## Present items as vectors

### Load data

In [1]:
from pandas import read_csv

data = read_csv("../input/rainbow-store/VinIDRecruitChallenge_MLTrack_DataSet.csv")
data

,csn,date,transaction_info
0,Y2NgaWJoYw==,2018-03-02,"[{'article': '10020163', 'salesquantity': 2.0,..."
1,Y2NgaWJoYw==,2018-03-04,"[{'article': '10026562', 'salesquantity': 3.0,..."
2,Y2NgaWlpYA==,2018-02-27,"[{'article': '10013531', 'salesquantity': 1.0,..."
3,Y2NgaWlpYA==,2018-03-16,"[{'article': '10008991', 'salesquantity': 1.0,..."
4,Y2NgaWlpYA==,2018-05-24,"[{'article': '10009698', 'salesquantity': 1.0,..."
...,...,...,...
131789,amxpamJk,2018-03-24,"[{'article': '10324916', 'salesquantity': 1.0,..."
131790,amxpamJk,2018-04-07,"[{'article': '10012785', 'salesquantity': 1.0,..."
131791,amxpamJk,2018-04-21,"[{'article': '10083185', 'salesquantity': 1.0,..."
131792,amxpamJk,2018-05-20,"[{'article': '10054789', 'salesquantity': 0.50..."


### Preprocess data

purchase list example

In [2]:
data['transaction_info'][1]

"[{'article': '10026562', 'salesquantity': 3.0, 'price': 13000.0}, {'article': '10320883', 'salesquantity': 2.0, 'price': 43000.0}]"

Convert transaction info to Python representation

In [3]:
from json import loads

data['info'] = [loads(js.replace("\'", "\"")) for js in data['transaction_info']]

List of rare items

In [4]:
lst = []
for transaction in data['info']:
    for item in transaction:
        lst.append(item['article'])
dic = {}
for item in lst:
    dic[item] = dic.get(item, 0) + 1
rare = []
for item in dic:
    if dic[item] < 5:
        rare.append(item)


In [5]:
len(lst)

993512

complete purchase list data

In [6]:
def convert(js):
    lst = []
    for item in js:
        article = item['article']
        if article in rare:
            lst.append('Rare')
        else:
            lst.append(article)
    return lst

In [7]:
purchase = [convert(js) for js in data['info']]

data comprehension

In [8]:
# maximum number of different items in a purchase
max([len(pur) for pur in purchase])

162

### Create items embedding

In [9]:
from gensim.models import Word2Vec

# used various window sizes, best value is 
model = Word2Vec(purchase, size=128, window=5, sg=1, hs=0, negative=5)

### Item clustering

In [10]:
from sklearn.cluster import OPTICS

vocab = model.wv.vocab
cluster = OPTICS(min_samples=6, metric='cosine', n_jobs=-1)
cluster.fit(model[vocab])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


OPTICS(metric='cosine', min_samples=6, n_jobs=-1)

In [11]:
len(set(cluster.labels_))

38

In [12]:
dic = {}
index = 0
for item in vocab:
    dic[item] = cluster.labels_[index]
    index += 1

### Save cluster group to file

In [13]:
from pickle import dump
with open("cluster.pkl","wb") as file:
    dump(dic, file, protocol=-1)